# Medium model (https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7)

# Environment Sound Classification Using a Two-Stream CNN Based on Decision-Level Fusion
 (https://docs.google.com/presentation/d/1M5D-h6AwXPxs7P_QaO7xW3yuaW6l7tQglIglTKNyAE4/edit?usp=sharing)

In [1]:
import matplotlib as mpl
mpl.use('Agg')

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import tensorflow as tf

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [5]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [6]:
import numpy as np

In [7]:
from keras import optimizers, Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPool2D
from keras.utils import np_utils, to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
# from keras.applications import DenseNet121

Using TensorFlow backend.


In [8]:
from sklearn import metrics, svm, datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder

In [89]:
#./Heart/CV_0001_0100/CV_0002/CV_0002_6.wav
#audio, sample_rate = librosa.load("./Heart/CV_0001_0100/CV_0001/CV_0001_1_1.wav", res_type='kaiser_fast')
audio, sample_rate = librosa.load("./Heart/CV_0001_0100/CV_0001/CV_0001_1_1.wav", res_type='kaiser_fast')
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate,n_mfcc=40)
print(mfccs.shape)
chromagram = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
print(chromagram.shape)
#S = np.abs(librosa.stft(audio))
#contrast = librosa.feature.spectral_contrast(S=S, sr=sample_rate)
#print(contrast.shape)
##T = librosa.effects.harmonic(audio)
#tonnetz = librosa.feature.tonnetz(y=T, sr=sample_rate)
#print(tonnetz.shape)

(40, 431)
(12, 431)


/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


## Function

In [90]:
def extract_features(file_name):
    print(file_name)
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate,n_mfcc=40)
        #print(mfccs.shape)
        chromagram = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        #print(chromagram.shape)
        S = np.abs(librosa.stft(audio))
        contrast = librosa.feature.spectral_contrast(S=S, sr=sample_rate)
        #print(contrast.shape)
        T = librosa.effects.harmonic(audio)
        tonnetz = librosa.feature.tonnetz(y=T, sr=sample_rate)
        #print(tonnetz.shape)
        
        all_1=np.append(mfccs,chromagram, axis=0)
        all_2=np.append(all_1,contrast, axis=0)
        all_3=np.append(all_2,tonnetz, axis=0)
        
    except Exception as e:
#         print("Error encountered while parsing file: ", file_name)
        return None 
    
    if all_3.shape == (0, 0):
        return none
    else:
        print(all_3.shape)
        return all_3

## Read Data

In [91]:
metadata = pd.read_csv("Heart/metadata.csv")

In [92]:
features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():    
    file_name = row["filename"]
    class_label = row["class"]
    data = extract_features(file_name)
    n = file_name.split("/")[-2]
    features.append([data, class_label, n])
featuresdf = pd.DataFrame(features, columns=['feature','class_label', 'N']).dropna()

Heart/CV_0001_0100/CV_0001/CV_0001_1_1.wav


/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


(65, 431)
Heart/CV_0001_0100/CV_0001/CV_0001_1_2.wav


/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


(65, 431)
Heart/CV_0001_0100/CV_0001/CV_0001_2_1.wav


...

(65, 431)
Heart/CV_0201_0300/CV_0206/CV_0206_5_1.WAV


/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


(65, 431)
Heart/CV_0201_0300/CV_0206/CV_0206_5_2.WAV


/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


(65, 431)
Heart/CV_0201_0300/CV_0206/CV_0206_6.WAV
(65, 431)


In [93]:
featuresdf.shape

(2264, 3)

In [63]:
featuresdf

,feature,class_label,N
0,"[[-456.69621553119316, -458.61692695603324, -4...",1,CV_0001
1,"[[-404.4839589469234, -412.39258884682926, -42...",1,CV_0001
2,"[[-452.465830397356, -458.57180343289235, -458...",2,CV_0001
3,"[[-452.63734929523946, -456.630662863603, -457...",2,CV_0001
4,"[[-417.57517553622887, -431.4601503948591, -46...",3,CV_0001
5,"[[-433.12030257254946, -440.24235403972887, -4...",3,CV_0001
6,"[[-443.4191356539869, -445.55438212841483, -44...",4,CV_0001
7,"[[-425.11248016710465, -433.28693854131353, -4...",4,CV_0001
8,"[[-375.7569212765933, -382.62885200108764, -39...",5,CV_0001
9,"[[-433.45598269418093, -441.383346318456, -448...",5,CV_0001


## Data Processing

In [94]:
N = np.array(list(set(featuresdf.N)))
mask = np.ones(N.shape,dtype=bool) #np.ones_like(a,dtype=bool)
mask[np.random.choice(len(N), int(len(N) * 0.9), replace=False)] = False

train = N[~mask]
test = N[mask]

featuresdf.index = list(featuresdf.N)

In [95]:
test

array(['CV_0175', 'CV_0047', 'CV_0172', 'CV_0184', 'CV_0193', 'CV_0201',
       'CV_0118', 'CV_0195', 'CV_0192', 'CV_0084', 'CV_0105', 'CV_0019',
       'CV_0114', 'CV_0200', 'CV_0043', 'CV_0085', 'CV_0150', 'CV_0120',
       'CV_0051', 'CV_0050', 'CV_0071'], dtype='<U7')

In [96]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.loc[train].feature.tolist())
y = np.array(featuresdf.loc[train].class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

x_train = X
y_train = yy
# x_train, _, y_train, _ = train_test_split(X, yy, test_size=0.1, random_state = 42)

In [97]:
y_train

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [98]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.loc[test].feature.tolist())
y = np.array(featuresdf.loc[test].class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 
x_test = X
y_test = yy
# x_test, _, y_test, _ = train_test_split(X, yy, test_size=0.0, random_state = 42)

## Init

In [100]:
num_rows = 65
num_columns = 431
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]

In [101]:
num_labels

6

In [102]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 64, 430, 32)       160       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 215, 32)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 215, 32)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 31, 214, 32)       4128      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 15, 107, 32)       0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 15, 107, 32)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 106, 64)       8256      
__________

Jay modify

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(2,2),strides=(2,2), input_shape=(num_rows, num_columns, num_channels), activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(2,2),strides=(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(2,2),strides=(2,2), activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(2,2),strides=(2,2), activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))
#model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(1024,activation='sigmoid'))
model.add(Dense(6, activation='softmax'))
model.summary()

## Training

In [103]:
# optimizer = optimizers.RMSprop() # IEEE
optimizer = optimizers.Adam() # Web, DenseNet
# optimizer.lr = 0.001
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)

model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Train on 2033 samples, validate on 231 samples
Epoch 1/100
2033/2033 [==============================] - 3s 1ms/step - loss: 2.2022 - acc: 0.2332 - val_loss: 1.6640 - val_acc: 0.2554
Epoch 2/100
2033/2033 [==============================] - 2s 885us/step - loss: 1.5591 - acc: 0.2907 - val_loss: 1.5419 - val_acc: 0.3030
Epoch 3/100
2033/2033 [==============================] - 2s 760us/step - loss: 1.5082 - acc: 0.2956 - val_loss: 1.5048 - val_acc: 0.3290
Epoch 4/100
2033/2033 [==============================] - 2s 902us/step - loss: 1.4629 - acc: 0.3227 - val_loss: 1.5074 - val_acc: 0.2641
Epoch 5/100
2033/2033 [==============================] - 1s 721us/step - loss: 1.4708 - acc: 0.3153 - val_loss: 1.5058 - val_acc: 0.2771
Epoch 6/100
2033/2033 [==============================] - 1s 732us/step - loss: 1.4786 - acc: 0.3158 - val_loss: 1.4882 - val_acc: 0.2814
Epoch 7/100
2033/2033 [==============================] - 1s 737us/step - loss: 1.4429 - acc: 0.3310 - val_loss: 1.4478 - val_acc: 0.3

## Confusion Matrix

In [104]:
preds = model.predict(x_test).argmax(axis=1)

In [105]:
y = y_test.argmax(axis=1)

In [106]:
confusion_matrix(y, preds)

array([[18,  9,  1,  1, 13,  0],
       [ 5, 12,  6,  5, 14,  0],
       [ 4, 12,  2,  9, 15,  0],
       [ 1,  6,  4, 10, 21,  0],
       [ 3,  6,  2,  4, 27,  0],
       [ 0,  0,  0,  0,  0, 21]])